Test queries on chicago_bikes database 

In [ ]:
import pymongo
from pymongo import MongoClient, UpdateOne
import json
# import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# Confirm list of databases
print(mongo.list_database_names())

In [ ]:
# Assign the database to a variable name
db = mongo.chicago_bikes

In [ ]:
# Review collections in database
print(db.list_collection_names())

In [ ]:
# Assign weather data to a variable 
weather_daily = db['weather_daily']

In [ ]:
# Count the number of documents in weather data 
print(weather_daily.count_documents({}))

In [ ]:
# Review a document in the weather_daily collection 
print(db.weather_daily.find_one())

In [ ]:
# Convert precipitation field value to inches (currently in mm)
# Conversion factor: 1 mm = 0.03937 inches
mm_to_inches = 0.03937

In [ ]:
# Iterate through documents and update values
for document in weather_daily.find():
    size_mm = document.get("precipitation")
    if size_mm is not None:
        size_inches = size_mm * mm_to_inches
        # Update the document with the new size value in inches
        weather_daily.update_one({"_id": document["_id"]}, {"$set": {"prcp_inches": size_inches}})

In [ ]:
# Review a document in the weather_daily collection to confirm update
print(db.weather_daily.find_one())

In [ ]:
# Drop 'size_inches' field as it has been replaced with 'prcp_inches' and values match 
# Specify the field you want to drop
field_to_drop = "size_inches"

In [ ]:
# Update documents to unset the specified field
weather_daily.update_many({}, {"$unset": {field_to_drop: 1}})

In [ ]:
# Review a document in the weather_daily collection to confirm drop
print(db.weather_daily.find_one())

In [ ]:
# Define the threshold value for significant precipitation comparison 
threshold = 0.1

In [ ]:
# Update documents to add the new field "sig_prcp"
# Add a new field 'sig_prcp' and populate 'yes' values if 'prcp_inches' is greather than or equal .1 inches
weather_daily.update_many(
    {"prcp_inches": {"$gte": threshold}},
    {"$set": {"sig_prcp": "yes"}}
)

weather_daily.update_many(
    {"prcp_inches": {"$lt": threshold}},
    {"$set": {"sig_prcp": "no"}}
)

In [ ]:
# Review a document in the weather_daily collection to confirm update
print(db.weather_daily.find_one())

In [ ]:
# Update documents to add the new field "avg_temp"
weather_daily.update_many(
    {},
    [
        {
            "$set": {
                "avg_temp": {
                    "$avg": ["$morning_temp", "$afternoon_temp", "$evening_temp"]
                }
            }
        }
    ]
)

In [ ]:
# Review a document in the weather_daily collection to confirm update
print(db.weather_daily.find_one())

In [ ]:
# Close the MongoDB connection
mongo.close()